In [ ]:
# https://www.lazard.com/research-insights/2023-levelized-cost-of-energyplus/

In [ ]:
%pip install pyomo

In [ ]:
import pyomo.environ as pyo

battery capacity less than or equal to off peak requirement

stored power = peak_requirement - solar_capacity

off peak cost = off peak grid price *  (off peak requirement - solar capacity)

peak cost = peak requirement - solar capacity

minimize total cost

In [59]:
model = pyo.ConcreteModel()
model.solar_capacity = pyo.Var(within=pyo.NonNegativeReals)
model.peak_grid_consumption = pyo.Var(within=pyo.NonNegativeReals)
model.off_peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals)
model.peak_load = pyo.Param(initialize=10)
model.off_peak_load = pyo.Param(initialize=5)
model.battery_capacity = pyo.Var(within=pyo.NonNegativeReals)


model.peak_grid_price = pyo.Param(initialize=0.2)
model.off_peak_grid_price = pyo.Param(initialize=0.5)
model.solar_cost_per_kw = pyo.Param(initialize=0.1)
model.battery_cost_per_kw = pyo.Param(initialize=0.1)

model.minimizeCost = pyo.Objective(
    expr=(model.peak_grid_price * model.peak_grid_consumption)
    + (model.off_peak_grid_price * model.off_peak_grid_usage)
    + (model.solar_cost_per_kw * model.solar_capacity)
    + (model.battery_cost_per_kw * model.battery_capacity),
    sense=pyo.minimize,
)
"""
model.abs_constraint1 = pyo.Constraint(
    expr=model.peak_load >= model.peak_grid_consumption - model.solar_capacity
)
model.abs_constraint2 = pyo.Constraint(
    expr=model.peak_load >= -(model.peak_grid_consumption - model.solar_capacity)
)
"""
model.off_peak_constraint = pyo.Constraint(
    expr=model.off_peak_load <= model.off_peak_grid_usage + model.battery_capacity
)

model.peak_constraint = pyo.Constraint(
    expr=model.peak_load <= model.peak_grid_consumption + model.solar_capacity
)

model.battery_charging_constraint = pyo.Constraint(
    expr=model.battery_capacity <= model.solar_capacity - model.peak_load
)


optimizer = pyo.SolverFactory("glpk")
optimizer.solve(model)
print(model.display())

Model unknown

  Variables:
    solar_capacity : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  15.0 :  None : False : False : NonNegativeReals
    peak_grid_consumption : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    off_peak_grid_usage : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    battery_capacity : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   5.0 :  None : False : False : NonNegativeReals

  Objectives:
    minimizeCost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   2.0

  Constraints:
    off_peak_constraint : Size=1
        Key  : Lower : Body : Upper
        None :   5.0 :  5.0 :  None
    peak_constr